In [107]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

data = pd.read_csv('Housing.csv')

In [108]:
# Перетворення категоріальних змінних на числові
data = pd.get_dummies(data, drop_first=True)

# Виведення кількох рядків для перевірки
print(data.head()) 

      price  area  bedrooms  bathrooms  stories  parking  mainroad_yes  \
0  13300000  7420         4          2        3        2          True   
1  12250000  8960         4          4        4        3          True   
2  12250000  9960         3          2        2        2          True   
3  12215000  7500         4          2        2        3          True   
4  11410000  7420         4          1        2        2          True   

   guestroom_yes  basement_yes  hotwaterheating_yes  airconditioning_yes  \
0          False         False                False                 True   
1          False         False                False                 True   
2          False          True                False                False   
3          False          True                False                 True   
4           True          True                False                 True   

   prefarea_yes  furnishingstatus_semi-furnished  furnishingstatus_unfurnished  
0          True  

In [109]:
X = data[['area', 'bathrooms', 'bedrooms']].values  
y = data['price'].values.reshape(-1, 1) 

In [110]:
# Стандартизація ознак
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X_std[X_std == 0] = 1  # Уникнення ділення на 0
X = (X - X_mean) / X_std

# Додавання стовпця одиниць до X
X = np.hstack([np.ones((X.shape[0], 1)), X])

In [111]:
# 1. Напишіть функцію гіпотези лінійної регресії у векторному вигляді
def hypothesis(X, theta):
    return X @ theta

# 2. Створіть функцію для обчислення функції втрат у векторному вигляді
def compute_loss(X, y, theta):
    m = len(y)
    predictions = hypothesis(X, theta)
    return (1 / (2 * m)) * np.sum((predictions - y) ** 2)

# 3. Реалізуйте один крок градієнтного спуску
def gradient_descent(X, y, theta, learning_rate, iterations):
    m = len(y)
    for _ in range(iterations):
        gradient = (1 / m) * X.T @ (hypothesis(X, theta) - y)
        theta -= learning_rate * gradient
    return theta

In [121]:
# 4. Знайдіть найкращі параметри θ для датасету, використовуючи написані функції
theta = np.zeros((X.shape[1], 1))
learning_rate = 0.01
iterations = 5000

# Градієнтний спуск
theta_gd = gradient_descent(X, y, theta, learning_rate, iterations)
print("Найкращі параметри θ (градієнтний спуск):", theta_gd.flatten())


Найкращі параметри θ (градієнтний спуск): [4766729.24770638  821214.14349519  695808.52272536  299983.57107964]


In [120]:
# 5. Знайдіть ці ж параметри за допомогою аналітичного рішення
theta_analytical = np.linalg.inv(X.T @ X) @ X.T @ y
print("Оптимальні параметри (аналітичне рішення):", theta_analytical.flatten())

Оптимальні параметри (аналітичне рішення): [4766729.24770642  821214.14349519  695808.52272537  299983.57107963]


In [114]:
# Прогнози
y_pred_gd = hypothesis(X, theta_gd)
y_pred_analytical = hypothesis(X, theta_analytical)

In [115]:
mse_gd = mean_squared_error(y, y_pred_gd)
mse_analytical = mean_squared_error(y, y_pred_analytical)

In [116]:
# 6. Для перевірки спрогнозованих значень використайте LinearRegression з бібліотеки scikit-learn
model = LinearRegression()
model.fit(X, y)  # Навчання моделі
y_pred_sklearn = model.predict(X)

# Порівняння результатів
mse_sklearn = mean_squared_error(y, y_pred_sklearn)
theta_sklearn = np.append(model.coef_, model.intercept_)

In [117]:
print("MSE (градієнтний спуск):", mse_gd)
print("MSE (аналітичне рішення):", mse_analytical)
print("MSE (scikit-learn):", mse_sklearn)

print("Параметри (градієнтний спуск):", theta_gd.flatten())
print("Параметри (аналітичне рішення):", theta_analytical.flatten())
print("Параметри (scikit-learn):", theta_sklearn.flatten())

MSE (градієнтний спуск): 1791170049977.3193
MSE (аналітичне рішення): 1791170049977.3193
MSE (scikit-learn): 1791170049977.319
Параметри (градієнтний спуск): [4766729.24770638  821214.14349519  695808.52272536  299983.57107964]
Параметри (аналітичне рішення): [4766729.24770642  821214.14349519  695808.52272537  299983.57107963]
Параметри (scikit-learn): [      0.          821214.14349519  695808.52272537  299983.57107963
 4766729.24770642]
